In [ ]:
import os
import nibabel as nib
import numpy as np

def resize_nifti(input_file, output_file, target_shape=(224, 224, 224), padding_value=-4000, background_threshold=-3000):
    # Load the NIfTI file
    img = nib.load(input_file)
    data = img.get_fdata()
    
    # Get the current shape
    current_shape = data.shape
    
    # Initialize the output array with the padding value
    output_data = np.full(target_shape, padding_value)
    
    # Dimension 0: Cropping and padding while preserving non-background voxels in [:, 0:100, :]
    if current_shape[0] > target_shape[0]:
        # Find the range of non-background voxels in [:, 0:100, :]
        non_bg_indices = np.where(data[:, 0:100, :] > background_threshold)[0]
        if len(non_bg_indices) > 0:
            start_preserve = non_bg_indices.min()
            end_preserve = non_bg_indices.max() + 1
            
            # Calculate potential crop
            total_crop = current_shape[0] - target_shape[0]
            start_crop = max(0, start_preserve - (total_crop // 2))
            end_crop = min(current_shape[0], start_crop + target_shape[0])
            
            # Adjust if end_crop removes non-background voxels
            if end_crop < end_preserve:
                end_crop = min(current_shape[0], end_preserve)
                start_crop = max(0, end_crop - target_shape[0])
            
            # Perform cropping
            dim0_data = data[start_crop:end_crop, :, :]
            
            # Pad if necessary (in case we couldn't crop fully)
            if dim0_data.shape[0] < target_shape[0]:
                pad_before = (target_shape[0] - dim0_data.shape[0]) // 2
                pad_after = target_shape[0] - dim0_data.shape[0] - pad_before
                dim0_data = np.pad(dim0_data, ((pad_before, pad_after), (0, 0), (0, 0)), 
                                   mode='constant', constant_values=padding_value)
        else:
            # If no non-background voxels, crop from center
            start = (current_shape[0] - target_shape[0]) // 2
            end = start + target_shape[0]
            dim0_data = data[start:end, :, :]
    else:
        # Pad if smaller
        pad_before = (target_shape[0] - current_shape[0]) // 2
        pad_after = target_shape[0] - current_shape[0] - pad_before
        dim0_data = np.pad(data, ((pad_before, pad_after), (0, 0), (0, 0)), 
                           mode='constant', constant_values=padding_value)

    # Dimension 1: Cropping and padding at the end
    if dim0_data.shape[1] > target_shape[1]:
        dim1_data = dim0_data[:, :target_shape[1], :]
    else:
        pad_width = target_shape[1] - dim0_data.shape[1]
        dim1_data = np.pad(dim0_data, ((0, 0), (0, pad_width), (0, 0)), 
                           mode='constant', constant_values=padding_value)

    # Dimension 2: Same as dimension 1
    if dim1_data.shape[2] > target_shape[2]:
        output_data = dim1_data[:, :, :target_shape[2]]
    else:
        pad_width = target_shape[2] - dim1_data.shape[2]
        output_data = np.pad(dim1_data, ((0, 0), (0, 0), (0, pad_width)), 
                             mode='constant', constant_values=padding_value)

    # Create a new NIfTI image with the resized data
    new_img = nib.Nifti1Image(output_data, img.affine, img.header)
    
    # Save the resized image
    nib.save(new_img, output_file)

# The process_folder function remains the same
def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.nii') or filename.endswith('.nii.gz'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            resize_nifti(input_path, output_path)
            print(f"Processed: {filename}")

# Usage
input_folder = r"C:\Users\acer\Desktop\Data_0\Nii_flipped"
output_folder = r"C:\Users\acer\Desktop\Data_0\Nii_cropped_224"
process_folder(input_folder, output_folder)